# Esercitazione 4 di Tecnologie del Linguaggio Naturale - Text categorization with Vector Space Models

Studenti:

- Brunello Matteo (mat. 858867)
- Caresio Lorenzo (mat. 836021)

## 1. Download e caricamento del dataset
Iniziamo scaricando ed estraendo il dataset dal provider (moodle).

Dopo essersi autenticati su moodle, inserire il valore del cookie `MoodleSession` (consultabile nella tab *Storage* nella finestra di ispeziona elemento su qualsiasi browser) all'interno della variabile `moodle_session_cookie` della cella seguente.

In [1]:
# Open the inspect element into your moodle session, then paste the "MoodleSession" field value in the storage/cookies tab
moodle_session_cookie = '87qml4svnpjvgvlh5frht45rtc'

!curl --cookie 'MoodleSession={moodle_session_cookie}' "https://informatica.i-learn.unito.it/pluginfile.php/364320/mod_folder/content/0/utils/data.zip?forcedownload=1" -o data.zip
!unzip -q data.zip "data/20_NGs_400/*" -x "data/20_NGs_400/.DS_Store"
!ls data/20_NGs_400

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1226k  100 1226k    0     0  3357k      0 --:--:-- --:--:-- --:--:-- 3369k
alt.atheism		  rec.autos	      sci.space
comp.graphics		  rec.motorcycles     soc.religion.christian
comp.os.ms-windows.misc   rec.sport.baseball  talk.politics.guns
comp.sys.ibm.pc.hardware  rec.sport.hockey    talk.politics.mideast
comp.sys.mac.hardware	  sci.crypt	      talk.politics.misc
comp.windows.x		  sci.electronics     talk.religion.misc
misc.forsale		  sci.med


Notiamo in primo luogo che il dataset è suddiviso per classi in diverse directories. Ogni directory contiene i documenti che fanno riferimento alla classe che rappresenta la directory stessa. Per caricare il dataset, si attraversa in una prima fase tutta la gerarchia delle directory, salvando durante il processo i vari path dei files e le loro classi corrispondenti. Durante questo processo vengono generati quindi 3 array:

* `paths`: contiene i path (completi) di tutti i file presenti nel dataset.
* `Y`: contiene le classi dei vari documenti.
* `Y_labels`: contiene i nomi delle classi (sono i nomi delle directories).

Una volta completata questa fase preliminare, si utilizza successivamente l'utility `TfidfVectorizer` della libreria `sklearn` per trasformare il vettore `paths` nei corrispondenti vettori con rappresentazione *Tfidf*.
I parametri del vettorizzatore sono:

* `input = 'filename'`: specifica che ogni riga della matrice passata in input rappresenta un path del documento che il vettorizzatore aprirà in lettura.
* `strip_accents = 'unicode'`: indica che la punteggiatura deve essere rimossa, così come i caratteri speciali.
* `stop_words = 'english'`: specifica che le stopwords devono essere eliminate. Esse sono determinate da un vocabolario nella lingua di riferimento dei documenti (in questo caso, Inglese).

Impostando questi parametri, l'utility si preoccuperà di aprire e leggere i vari files dei documenti, rimuovendo punteggiatura e stopwords, calcolando infine le frequenze necessarie per ottenere la rappresentazione vettoriale desiderata.

Una volta ottenuta questa nuova rappresentazione, la procedura di caricamento può terminare correttamente, ritornando il dataset in formato *Tfidf*.

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import numpy as np

# The idea is to first traverse the entire dataset's hierarchy while keeping
# track of classes and the corresponding paths of each document. In this way,
# we can then use the list of paths to create (Tf * Idf) weight vectors for each
# document (in the same order on which we visited the dataset), just by using
# the sklearn's utility TfIdfVectorizer
def load_tfidf_dataset(base_dir: str):
  X, Y, paths = [], [], []
  # y labels are just going to be directory names
  Y_labels = np.array(os.listdir(base_dir))
  for c_i, class_name in enumerate(Y_labels):
    # get the documents of that class
    for document in os.listdir(f'{base_dir}/{class_name}'):
      Y.append(c_i)                                       # append the class index
      paths.append(f'{base_dir}/{class_name}/{document}') # append the path of the document (needed by the TfidfVectorizer)
  # transform Y into a column vector
  Y = np.array(Y)
  # build and fit the vectorizer
  vectorizer = TfidfVectorizer(
      input = 'filename',
      strip_accents = 'unicode',
      stop_words = 'english'
  )
  # create the weight vectors and transform to a normal array (since it returns a sparse representation matrix)
  X = vectorizer.fit_transform(paths).toarray()
  return X, Y, Y_labels

## 2. Definizione del modello
Definiamo ora il modello di Rocchio per fare classificazione. Come ogni modello di machine learning, esso avrà degli iperparametri definibili dall'utente e dei parametri che verranno invece appresi durante la fase di learning.

Nel nostro caso, gli iperparametri possono essere impostati attraverso il costruttore della classe e sono:

* $\gamma$ e $\beta$: si veda la formula citata più avanti.
* `threshold` ($t$): se specificato, il training considererà i near positives come insieme di negativi, altrimenti l'intero set dei negativi verrà considerato (*ulteriori dettagli verranno discussi successivamente*).
* `similarity`: funzione di similarità che deve utilizzare il modello. Di default è la *Cosine Similarity*.

L'implementazione segue a grandi linee le scelte di design della libreria `sklearn`, per cui il modello mette a disposizione 2 metodi:

* `fit`: apprende i parametri del modello sulla base dei dati passati in input.
* `__call__`: ritorna la classe corrispondente al profilo di Rocchio più vicino all'esempio passato in input. L'override dell'operatore di *call* permette di chiamare il classificatore come se fosse una funzione, che ne aumenta la leggibilità del codice.

La fase di apprendimento si occupa di calcolare i parametri del modello, che corrispondono ai profili di Rocchio per ogni classe. Questi profili sono appresi per mezzo dell'implementazione diretta della seguente formula (in caso l'iperparametro `threshold` non sia specificato):

$$
f_{ki} = \beta \cdot \sum_{d_j \in POS_i} \frac{w_{kj}}{|POS_i|} - \gamma \cdot \sum_{d_j \in NEG_i} \frac{w_{kj}}{|NEG_i|}
$$

Altrimenti, per mezzo dell'implementazione della seguente:

$$
f_{ki} = \beta \cdot \sum_{d_j \in POS_i} \frac{w_{kj}}{|POS_i|} - \gamma \cdot \sum_{d_j \in NPOS_i} \frac{w_{kj}}{|NPOS_i|}
$$

Nel caso della seconda, i *near positives* sono definiti come
$$
NPOS_i = \{ \vec{w} \in NEG_i  \; : \; sim(\vec{w}, \vec{p}) \geq t\}
$$
dove:

* $NEG_i$ è l'insieme dei vettori della classe negativa.
* $\vec{p}$ è il centroide della classe positiva.
* $t$ è un iperparametro di threshold.
* $sim(x, y)$ è la funzione che ritorna la similarità tra $x$ e $y$.

Per effettuare i calcoli necessari in modo efficiente, si è optato di utilizzare la libreria per il cacolo numerico `numpy`. Vettorizzando efficientemente le varie operazioni, la fase di learning può sfruttare le varie ottimizzazioni messe in atto dalla libreria, tra cui l'utilizzo delle routine SIMD per l'accelerazione del calcolo.

La fase di prediction, invece, consiste semplicemente nel calcolare la distanza del vettore in input (che rappresenta un documento) con tutti i profili di Rocchio appresi nella fase di learning. Successivamente, si ritorna la classe associata al profilo di Rocchio più prossimo al vettore in input (*highest rank*). Dal punto di vista implementativo, si è utilizzato il [broadcasting](https://numpy.org/doc/stable/user/basics.broadcasting.html) in modo da sfruttare le ottimizzazioni di `numpy`. Inoltre, l'impiego di routine di `numpy` permette di fare prediction su un batch di vettori con una sola operazione in modo molto efficiente.

In [3]:
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity

class RocchioClassifier:

  def __init__(self, beta = 16, gamma = 4, threshold = None, similarity = cosine_similarity):
    self.beta = beta
    self.gamma = gamma
    self.profiles = None
    self.similarity = similarity
    self.threshold = threshold

  # Build the Rocchio profiles for each class in the dataset
  def fit(self, X, Y):
    # get the number of classes
    profiles = []
    for c_i in np.unique(Y):
      # take the set of positives and negatives by comparing their class with the current class
      pos, neg = X[Y == c_i], X[Y != c_i]
      # compute the positive centroid
      p_centroid = np.mean(pos, axis = 0)
      # if the threshold is defined, then we need to use the near-positives formulation
      # for the negative samples
      if self.threshold:
        # first compute the similarities between each negative sample and
        # the positive centroid
        similarities = self.similarity(neg, p_centroid.reshape(1, -1)).flatten()
        # we re-define the negatives as all the documents that are at least similar
        # according to a minimum threshold value
        neg = neg[similarities >= self.threshold]
      # compute the negative centroid
      n_centroid = np.mean(neg, axis = 0)
      # add the current rocchio profile to the array of classes
      profile = (self.beta * p_centroid) - (self.gamma * n_centroid)
      profiles.append(profile)
    # set the learned profiles as the previously computed profiles
    self.profiles = np.array(profiles)

  # predict the most similar class given a vector (or a series of vectors)
  def __call__(self, X):
    # compute all similarities between each input vector and each class
    similarities = self.similarity(X, self.profiles)
    # return the most similar class for each input
    return np.argmax(similarities, axis = 1)

## 3. Valutazione delle performance
Una volta ottenuto il modello è necessario valutarne le performance. Una tecnica molto nota nel machine learning è il *K-fold Validation*, in cui si utilizzano diversi split *train-test* (chiamati *fold*) del dataset valutandone prima le performance sul singolo fold, per poi fare una media totale dei risultati associati ai vari fold.

Per calcolare i vari fold, è stata utilizzata l'utility `KFold` di `sklearn`. Essa ritorna una lista di tuple contenenti due liste di indici associati agli elementi che rappresentano rispettivamente il *training set* e il *test set*.

L'implementazione della procedura a questo punto consiste nell'iterare i vari fold, recuperando inizialmente gli elementi nel dataset corrispondenti agli indici. Successivamente si apprende il modello sui dati di train del fold, per poi fare prediction e infine calcolare l'accuracy del fold considerato.
I vari risultati sono accumulati in una variabile, che viene poi divisa per il numero di fold per ottenere la media totale.

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

def kfold_eval(folds, model = RocchioClassifier()):
  kfold = KFold(n_splits = folds, shuffle=True, random_state = 42)
  print(f'\u001b[36m------ Evaluating over {folds} folds  ------\u001b[0m')
  print('\tK-Fold Nr. \t Accuracy')
  print('\u001b[36m---------------------------------------\u001b[0m')
  mean_acc = 0
  for i, (train_idx, test_idx) in enumerate(kfold.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    # train the model
    model.fit(X_train, Y_train)
    # evaluate the accuracy of the model
    Y_pred = model(X_test)
    accuracy = accuracy_score(Y_test, Y_pred)
    print(f'\t {i+1} \t\t {accuracy}')
    mean_acc += accuracy
  # compute the mean accuracy over all folds
  print('\u001b[36m---------------------------------------\u001b[0m')
  print(f'Mean Accuracy over {folds} folds: {mean_acc / folds}%')
  print('\u001b[36m---------------------------------------\u001b[0m\n')

Una volta implementata la funzione, è sufficiente caricare il dataset con l'utility apposita, definire il modello e richiamare la funzione atta a fare *K-fold validation*.

In [5]:
dataset_dir = 'data/20_NGs_400'

# Load the dataset
X, Y, Y_l = load_tfidf_dataset(dataset_dir)
folds = 10
# Evaluate the first model using the k-fold evaluation method
model = RocchioClassifier()
kfold_eval(folds, model)
# Evaluate the second model using the k-fold evaluation method, using a threshold of 0.2
model = RocchioClassifier(threshold = 0.02)
kfold_eval(folds, model)

------ Evaluating over 10 folds  ------
	K-Fold Nr. 	 Accuracy
---------------------------------------
	 1 		 0.725
	 2 		 0.65
	 3 		 0.65
	 4 		 0.75
	 5 		 0.8
	 6 		 0.6
	 7 		 0.575
	 8 		 0.7
	 9 		 0.575
	 10 		 0.75
---------------------------------------
Mean Accuracy over 10 folds: 0.6775%
---------------------------------------

------ Evaluating over 10 folds  ------
	K-Fold Nr. 	 Accuracy
---------------------------------------
	 1 		 0.725
	 2 		 0.65
	 3 		 0.625
	 4 		 0.75
	 5 		 0.8
	 6 		 0.6
	 7 		 0.575
	 8 		 0.7
	 9 		 0.575
	 10 		 0.75
---------------------------------------
Mean Accuracy over 10 folds: 0.675%
---------------------------------------

